# Get Multi-Variate Time Series Data

In [1]:
!wget https://s3-us-west-2.amazonaws.com/telemanom/data.zip

--2023-06-09 16:06:32--  https://s3-us-west-2.amazonaws.com/telemanom/data.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.131.32, 3.5.80.142, 52.92.145.232, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.131.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85899803 (82M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  81.92M  34.7MB/s    in 2.4s    

2023-06-09 16:06:35 (34.7 MB/s) - ‘data.zip’ saved [85899803/85899803]



In [2]:
!unzip -qq data.zip

In [3]:
!ls data

2018-05-19_15.00.10  test  train


In [4]:
!ls data/train

A-1.npy  C-2.npy   D-5.npy   E-3.npy  F-5.npy  M-3.npy	 P-3.npy   T-3.npy
A-2.npy  D-11.npy  D-6.npy   E-4.npy  F-7.npy  M-4.npy	 P-4.npy   T-4.npy
A-3.npy  D-12.npy  D-7.npy   E-5.npy  F-8.npy  M-5.npy	 P-7.npy   T-5.npy
A-4.npy  D-13.npy  D-8.npy   E-6.npy  G-1.npy  M-6.npy	 R-1.npy   T-8.npy
A-5.npy  D-14.npy  D-9.npy   E-7.npy  G-2.npy  M-7.npy	 S-1.npy   T-9.npy
A-6.npy  D-15.npy  E-10.npy  E-8.npy  G-3.npy  P-10.npy  S-2.npy
A-7.npy  D-16.npy  E-11.npy  E-9.npy  G-4.npy  P-11.npy  T-10.npy
A-8.npy  D-1.npy   E-12.npy  F-1.npy  G-6.npy  P-14.npy  T-12.npy
A-9.npy  D-2.npy   E-13.npy  F-2.npy  G-7.npy  P-15.npy  T-13.npy
B-1.npy  D-3.npy   E-1.npy   F-3.npy  M-1.npy  P-1.npy	 T-1.npy
C-1.npy  D-4.npy   E-2.npy   F-4.npy  M-2.npy  P-2.npy	 T-2.npy


In [5]:
!ls data/test

A-1.npy  C-2.npy   D-5.npy   E-3.npy  F-5.npy  M-3.npy	 P-3.npy   T-3.npy
A-2.npy  D-11.npy  D-6.npy   E-4.npy  F-7.npy  M-4.npy	 P-4.npy   T-4.npy
A-3.npy  D-12.npy  D-7.npy   E-5.npy  F-8.npy  M-5.npy	 P-7.npy   T-5.npy
A-4.npy  D-13.npy  D-8.npy   E-6.npy  G-1.npy  M-6.npy	 R-1.npy   T-8.npy
A-5.npy  D-14.npy  D-9.npy   E-7.npy  G-2.npy  M-7.npy	 S-1.npy   T-9.npy
A-6.npy  D-15.npy  E-10.npy  E-8.npy  G-3.npy  P-10.npy  S-2.npy
A-7.npy  D-16.npy  E-11.npy  E-9.npy  G-4.npy  P-11.npy  T-10.npy
A-8.npy  D-1.npy   E-12.npy  F-1.npy  G-6.npy  P-14.npy  T-12.npy
A-9.npy  D-2.npy   E-13.npy  F-2.npy  G-7.npy  P-15.npy  T-13.npy
B-1.npy  D-3.npy   E-1.npy   F-3.npy  M-1.npy  P-1.npy	 T-1.npy
C-1.npy  D-4.npy   E-2.npy   F-4.npy  M-2.npy  P-2.npy	 T-2.npy


In [6]:
!git clone https://github.com/ML4ITS/mtad-gat-pytorch.git

Cloning into 'mtad-gat-pytorch'...
remote: Enumerating objects: 6189, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 6189 (delta 11), reused 0 (delta 0), pack-reused 6161
Receiving objects: 100% (6189/6189), 920.67 MiB | 42.20 MiB/s, done.
Resolving deltas: 100% (2708/2708), done.
Updating files: 100% (158/158), done.


In [7]:
!ls mtad-gat-pytorch/datasets/data

labeled_anomalies.csv  msl_train_md.csv  smap_train_md.csv


# Import Libraries

In [8]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.nn as nn
from tqdm.notebook import tqdm


In [9]:
smap_channels = pd.read_csv('mtad-gat-pytorch/datasets/data/smap_train_md.csv')
print("Number of Channels : ",len(smap_channels))
smap_channels.head(5)

Number of Channels :  53


,chan_id,num_values
0,A-1,2880
1,A-2,2648
2,A-3,2736
3,A-4,2690
4,A-5,705


In [10]:
smap_channels = list(smap_channels['chan_id'].values)
len(smap_channels)

53

# Create SMAP Data from Time Series across all the Channel IDs - Each Time Step has 25 features

In [11]:
smap_data =[]

for smap_channel in smap_channels:
  tmp_data = np.load(os.path.join('data/train/',smap_channel+'.npy'))
  smap_data.extend(tmp_data)
  #print(smap_data.shape)

smap_data = np.array(smap_data)
print("Shape of SMAP Data : ", smap_data.shape)

Shape of SMAP Data :  (135183, 25)


In [12]:
smap_data

array([[0.999     , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.999     , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.999     , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98775593, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98417906, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98417906, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [13]:
# Next the Data is scaled using Min-Max Scaler

In [14]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [15]:
def normalize_data(data, scaler=None):
  data = np.asarray(data, dtype=np.float32)
  if np.any(sum(np.isnan(data))):
    data = np.nan_to_num(data)
  
  if scaler is None:
    scaler=MinMaxScaler()
    scaler.fit(data)
  data=scaler.transform(data)
  print("Data normalized")

  return data, scaler

In [16]:
smap_data_norm, scaler = normalize_data(smap_data)

Data normalized


In [17]:
smap_data_pt = torch.from_numpy(smap_data)
smap_data_pt.size()

torch.Size([135183, 25])

# Create Sliding Window Dataset - Breaking down the entire time series into small temporal segments which will be used to train the model.

In [18]:
class SlidingWindowDataset(Dataset):
  def __init__(self, data, window, target_dim=None, horizon=1):
    self.data= data
    self.window = window
    self.target_dim = target_dim
    self.horizon = horizon

  def __getitem__(self,index):
    x = self.data[index : index+self.window]
    y = self.data[index + self.window : index + self.window + self.horizon]
    return x,y

  def __len__(self):
    return len(self.data) - self.window

In [19]:
Window=100
BatchSZ=256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
smap_x_y = SlidingWindowDataset(smap_data_pt, 100)

In [21]:
smap_x_y

In [22]:
def create_data_loaders(train_dataset, batch_size, val_split=0.1, shuffle=True, test_dataset=None):
    train_loader, val_loader, test_loader = None, None, None
    #if we set ratio 0
    if val_split == 0.0:
        print(f"train_size: {len(train_dataset)}")
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

    else:
        dataset_size = len(train_dataset)
        indices = list(range(dataset_size))
        split = int(np.floor(val_split * dataset_size))
        if shuffle:
            np.random.shuffle(indices)
        train_indices, val_indices = indices[split:], indices[:split]

        train_sampler = SubsetRandomSampler(train_indices)
        valid_sampler = SubsetRandomSampler(val_indices)

        # in torch DataLoader we need to set batch_size but, I'm not sure to set 
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler)

        print(f"train_size: {len(train_indices)}")
        print(f"validation_size: {len(val_indices)}")

    if test_dataset is not None:
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        print(f"test_size: {len(test_dataset)}")
    #if test_dataset is None then we return None?
    return train_loader, val_loader, test_loader

In [23]:
train_dl, val_dl, _ = create_data_loaders(smap_x_y, BatchSZ)

train_size: 121575
validation_size: 13508


- Until now, smap_x_y : is 
1. slided
2. torched from numpy
3. extended data -> np.array()

In [24]:
val_dl

> We determine the shape of batch
> - X : 256 batch size, 100 window length, 25 #Features

> - Y : 256 bathc size, 1 future value, 25 #Features.

In [25]:
eg = next(iter(val_dl))
eg[0].size(), eg[1].size()

(torch.Size([256, 100, 25]), torch.Size([256, 1, 25]))

In [26]:
import sys
sys.path.insert(0,'mtad-gat-pytorch/')

In [27]:
from modules import (
    ConvLayer,
    FeatureAttentionLayer,
    TemporalAttentionLayer,
    GRULayer,
    Forecasting_Model,
    ReconstructionModel,
)

In [28]:
class MTAD_GAT(nn.Module):

    def __init__(
        self,
        n_features,
        window_size,
        out_dim,
        kernel_size=7,
        feat_gat_embed_dim=None,
        time_gat_embed_dim=None,
        use_gatv2=True,
        gru_n_layers=1,
        gru_hid_dim=150,
        forecast_n_layers=1,
        forecast_hid_dim=150,
        recon_n_layers=1,
        recon_hid_dim=150,
        dropout=0.2,
        alpha=0.2
    ):
        super(MTAD_GAT, self).__init__()

        self.conv = ConvLayer(n_features, kernel_size)
        self.feature_gat = FeatureAttentionLayer(n_features, window_size, dropout, alpha, feat_gat_embed_dim, use_gatv2, use_bias = False)
        self.temporal_gat = TemporalAttentionLayer(n_features, window_size, dropout, alpha, time_gat_embed_dim, use_gatv2, use_bias = False)
        self.gru = GRULayer(3 * n_features, gru_hid_dim, gru_n_layers, dropout)
        self.forecasting_model = Forecasting_Model(gru_hid_dim, forecast_hid_dim, out_dim, forecast_n_layers, dropout)
        self.recon_model = ReconstructionModel(window_size, gru_hid_dim, recon_hid_dim, out_dim, recon_n_layers, dropout)

    def forward(self, x):
        # x shape (b, n, k): b - batch size, n - window size, k - number of features

        x = self.conv(x)
        h_feat = self.feature_gat(x)
        h_temp = self.temporal_gat(x)

        h_cat = torch.cat([x, h_feat, h_temp], dim=2)  # (b, n, 3k)

        _, h_end = self.gru(h_cat)
        h_end = h_end.view(x.shape[0], -1)   # Hidden state for last timestamp

        predictions = self.forecasting_model(h_end)
        recons = self.recon_model(h_end)

        return predictions, recons

In [29]:
model = MTAD_GAT(n_features = 25, window_size = 100, out_dim = 25, feat_gat_embed_dim = 256, time_gat_embed_dim = 64)

In [30]:
model.to(device)

MTAD_GAT(
  (conv): ConvLayer(
    (padding): ConstantPad1d(padding=(3, 3), value=0.0)
    (conv): Conv1d(25, 25, kernel_size=(7,), stride=(1,))
    (relu): ReLU()
  )
  (feature_gat): FeatureAttentionLayer(
    (lin): Linear(in_features=200, out_features=512, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (temporal_gat): TemporalAttentionLayer(
    (lin): Linear(in_features=50, out_features=128, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (gru): GRULayer(
    (gru): GRU(75, 150, batch_first=True)
  )
  (forecasting_model): Forecasting_Model(
    (layers): ModuleList(
      (0): Linear(in_features=150, out_features=150, bias=True)
      (1): Linear(in_features=150, out_features=25, bias=True)
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (relu): ReLU()
  )
  (recon_model): ReconstructionModel(
    (decoder): RNNDecoder(
      (rnn): GRU(150, 150, batch_first=True)
    )
    (fc): Linear(in_f

In [31]:
eg_out = model(eg[0].float().to(device))
# First Entity : Future Time Step Forecasted
# Second Entity : Reconstructed original input
eg_out[0].size(), eg_out[1].size()

(torch.Size([256, 25]), torch.Size([256, 100, 25]))

In [32]:
# Optimizer : Adam lr = 0.001 we use the MSE maybe
# Both Forecasting and Reconstruction would use MSE Loss function
EPOCHS = 2
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
forecast_criterion = nn.MSELoss()
recon_criterion = nn.MSELoss()

In [33]:
len(train_dl), len(val_dl)

(475, 53)

# Training Loop, Final Loss = Forecasting Loss + Reconstruction Loss, Track Loss on Eval set

In [34]:
!mkdir model_checkpoints

In [35]:
Best_Valid_loss = 999999
Best_Epoch=-1

for epoch in range(EPOCHS):
  model.train()
  forecast_b_losses = []
  recon_b_losses = []

  print("Epoch : ",epoch)
  print("Training Started ... ")

  for batch_idx, batch in enumerate(train_dl):
    x = batch[0].to(device).float()
    y = batch[1].to(device).float()
    optimizer.zero_grad()

    preds, recons = model(x)

    forecast_loss = torch.sqrt(forecast_criterion(y.squeeze(1),preds))
    recon_loss = torch.sqrt(recon_criterion(x, recons))

    loss = forecast_loss + recon_loss
    loss.backward()
    optimizer.step()

    forecast_b_losses.append(forecast_loss.item())
    recon_b_losses.append(recon_loss.item())

  forecast_b_losses = np.array(forecast_b_losses)
  recon_b_losses = np.array(recon_b_losses)

  forecast_epoch_loss = np.sqrt((forecast_b_losses ** 2).mean())
  recon_epoch_loss = np.sqrt((recon_b_losses**2).mean())
  total_epoch_loss = forecast_epoch_loss + recon_epoch_loss

  print('Forecasting Loss : ', forecast_epoch_loss)
  print('Reconstruction Loss : ', recon_epoch_loss)

  forecast_b_losses_eval = []
  recon_b_losses_eval = []

  print("Validation Started ... ")

  model.eval()
  with torch.no_grad():
    for batch_idx, batch in enumerate(val_dl):
      x = batch[0].to(device).float()
      y = batch[1].to(device).float()

      preds , recons = model(x)

      forecast_loss = torch.sqrt(forecast_criterion(y.squeeze(1), preds))
      recon_loss = torch.sqrt(recon_criterion(x,recons))
      loss = forecast_loss + recon_loss

      forecast_b_losses_eval.append(forecast_loss.item())
      recon_b_losses_eval.append(recon_loss.item())
  
  forecast_b_losses_eval = np.array(forecast_b_losses_eval)
  recon_b_losses_eval = np.array(recon_b_losses_eval)

  forecast_epoch_loss_eval = np.sqrt((forecast_b_losses_eval**2).mean())
  recon_epoch_loss_eval = np.sqrt((recon_b_losses_eval**2).mean())

  total_epoch_loss_eval = forecast_epoch_loss_eval + recon_epoch_loss_eval

  print('Forecasting Loss : ', forecast_epoch_loss_eval)
  print('Reconstruction Loss : ', recon_epoch_loss_eval)

  if total_epoch_loss_eval < Best_Valid_loss:
    Best_Valid_loss = total_epoch_loss_eval
    Best_Epoch = epoch

    ckpt = {
        'Epoch' : epoch,
        'Model' : model.state_dict(),
        'Optimizer' : optimizer.state_dict(),
        'Train_Forecast_loss' : forecast_epoch_loss,
        'Train_Recon_loss' : recon_epoch_loss,
        'Eval_Forecast_loss' : forecast_epoch_loss_eval,
        'Eval_Recon' : recon_epoch_loss_eval
    }

    torch.save(ckpt, os.path.join('model_checkpoints', str(epoch) + '.pt'))
    






Epoch :  0
Training Started ... 
Forecasting Loss :  0.10421131647579025
Reconstruction Loss :  0.13184312670454768
Validation Started ... 
Forecasting Loss :  0.0855248189424488
Reconstruction Loss :  0.10180436471586911
Epoch :  1
Training Started ... 
Forecasting Loss :  0.08638069228214976
Reconstruction Loss :  0.09682324691539598
Validation Started ... 
Forecasting Loss :  0.0816673492316436
Reconstruction Loss :  0.09119145062825904


In [36]:
best_model = MTAD_GAT(n_features = 25, window_size=100, out_dim=25, feat_gat_embed_dim = 256, time_gat_embed_dim = 64)
best_model.load_state_dict(torch.load(os.path.join('model_checkpoints', str(Best_Epoch)+'.pt'))['Model'])
best_model.to(device)




MTAD_GAT(
  (conv): ConvLayer(
    (padding): ConstantPad1d(padding=(3, 3), value=0.0)
    (conv): Conv1d(25, 25, kernel_size=(7,), stride=(1,))
    (relu): ReLU()
  )
  (feature_gat): FeatureAttentionLayer(
    (lin): Linear(in_features=200, out_features=512, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (temporal_gat): TemporalAttentionLayer(
    (lin): Linear(in_features=50, out_features=128, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
    (sigmoid): Sigmoid()
  )
  (gru): GRULayer(
    (gru): GRU(75, 150, batch_first=True)
  )
  (forecasting_model): Forecasting_Model(
    (layers): ModuleList(
      (0): Linear(in_features=150, out_features=150, bias=True)
      (1): Linear(in_features=150, out_features=25, bias=True)
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (relu): ReLU()
  )
  (recon_model): ReconstructionModel(
    (decoder): RNNDecoder(
      (rnn): GRU(150, 150, batch_first=True)
    )
    (fc): Linear(in_f

# Score the Losses on Test Dataset

In [37]:
smap_data_test = []
for smap_channel in smap_channels:
  tmp_data = np.load(os.path.join('data/test/', smap_channel + '.npy'))
  smap_data_test.extend(tmp_data)

smap_data_test = np.array(smap_data_test)
print("Shape of Test DataSet : ", smap_data_test.shape)

Shape of Test DataSet :  (427617, 25)


In [38]:
smap_data_test_norm, _ = normalize_data(smap_data_test, scaler)
smap_data_test_norm = smap_data_test_norm[:10000]
smap_data_test_pt = torch.from_numpy(smap_data_test_norm)
smap_data_test_pt.size()

Data normalized


torch.Size([10000, 25])

In [39]:
smap_test_x_y = SlidingWindowDataset(smap_data_test_pt,100)
test_dl = torch.utils.data.DataLoader(smap_test_x_y, batch_size=256, shuffle=False)
print("Number of Batches in Test Dataloader : ", len(test_dl))

Number of Batches in Test Dataloader :  39


# For Every time steps, Get the Reconstruction & Forecast Loss.

# E.g. - If window size = 10, For TS11 - To get Forecasting Loss, we input(TS1 TS2 ... TS10) in model & to get reconstruction loss, input(TS2 TS3 .. TS11)

In [60]:
model.eval()
preds = []
#preds=np.empty((1,2))
recons = []
#recons=np.empty((1,2))
with torch.no_grad():
  for batch in tqdm(test_dl):
    x = batch[0].to(device).float()
    y = batch[1].to(device).float()

    y_hat, _ =model(x)

    recon_x = torch.cat((x[:,1:,:],y),dim=1)
    _, window_recon = model(recon_x)
    print(window_recon[:,-1,:].detach().cpu().numpy().shape, y_hat.detach().cpu().numpy().shape)
    preds.append(y_hat.detach().cpu().numpy())
    #preds=np.append(preds,y_hat.detach().cpu().numpy())
    
    #preds = np.concatenate((preds, y_hat.detach().cpu().numpy()),axis=0)
    # Extract last reconstruction only
    recons.append(window_recon[:,-1,:].detach().cpu().numpy())
    #recons=np.append(recons,window_recon[:,-1,:].detach().cpu().numpy())
    #recons = np.concatenate((recons,window_recon[:,-1,:].detach().cpu().numpy()),axis=0)
    #print(preds,recons)
  preds = np.concatenate(preds, axis=0)
  recons = np.concatenate(recons,axis=0)

  0%|          | 0/39 [00:00<?, ?it/s]

(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(256, 25) (256, 25)
(172, 25) (172, 25)


- pred array contains Forecasting loss for all the time steps & all features in test data while recons array contains Reconstruction Loss

In [ ]:
#preds = np.concatenate([preds], axis=0)
#recons = np.concatenate([recons],axis=0)

In [61]:
preds.shape, recons.shape

((9900, 25), (9900, 25))

In [62]:
scale_scores= True

# Put the losses in a Dataframe

In [64]:
actual = smap_data_test_norm[Window:]
print(actual.shape)

anomaly_scores = np.zeros_like(actual)
df = pd.DataFrame()
for i in range(preds.shape[1]):
  df[f"Forecast_{i}"] = preds[:,i]
  df[f"Recon_{i}"] = recons[:,i]
  df[f"True_{i}"] = actual[:,i]
  a_score = np.sqrt((preds[:,i] - actual[:,i])**2) + np.sqrt((recons[:,i] - actual[:,i])**2)

  if scale_scores:
    q75, q25 = np.percentile(a_score, [75,25])
    iqr = q75 - q25
    median = np.median(a_score)
    a_score = (a_score - median)/(1+iqr)
  anomaly_scores[:,i] = a_score
  df[f"A_Score_{i}"] = a_score
anomaly_scores = np.mean(anomaly_scores, 1)
df['A_Score_Global'] = anomaly_scores

(9900, 25)


<ipython-input-64-5dc2ce0dba25>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['A_Score_Global'] = anomaly_scores



# Forecast{i}, Recon{i} = Forecast and Reconstruction loss for the i'th feature of the time step, A Score{i} = Forecast+Recon loss



# A_Score_Global : Mean of all loss across all features in that time step

In [65]:
df

,Forecast_0,Recon_0,True_0,A_Score_0,Forecast_1,Recon_1,True_1,A_Score_1,Forecast_2,Recon_2,...,A_Score_22,Forecast_23,Recon_23,True_23,A_Score_23,Forecast_24,Recon_24,True_24,A_Score_24,A_Score_Global
0,0.990661,0.983908,1.000000,-0.025418,0.007426,-0.005951,0.0,-0.006980,-0.000326,-0.000678,...,-0.003078,0.000325,0.002116,0.0,0.000082,-0.000048,0.002371,0.0,-0.000461,-0.002537
1,0.993305,0.983921,1.000000,-0.027817,0.008087,-0.005970,0.0,-0.006332,-0.000657,-0.000852,...,-0.002446,0.000341,0.002084,0.0,0.000066,-0.000051,0.002401,0.0,-0.000429,-0.002528
2,0.994401,0.983343,1.000000,-0.028284,0.010532,-0.006012,0.0,-0.003956,-0.000422,-0.000981,...,-0.001836,0.000290,0.002089,0.0,0.000020,-0.000045,0.002400,0.0,-0.000436,-0.002235
3,0.997320,0.983095,1.000000,-0.030696,0.012976,-0.006033,0.0,-0.001602,-0.000084,-0.001055,...,-0.001276,0.000222,0.002085,0.0,-0.000051,-0.000037,0.002378,0.0,-0.000466,-0.002056
4,0.998966,0.982511,1.000000,-0.031656,0.015931,-0.006043,0.0,0.001230,0.000234,-0.001100,...,-0.000620,0.000138,0.002095,0.0,-0.000126,-0.000028,0.002377,0.0,-0.000476,-0.001758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9895,0.212664,0.025773,0.144039,0.120369,-0.059610,0.258013,0.0,0.283575,-0.000321,0.005369,...,0.013354,-0.000127,-0.004091,0.0,0.001855,-0.000450,-0.006210,0.0,0.003767,0.051107
9896,0.166093,0.144286,0.144039,-0.028244,-0.026688,0.101545,0.0,0.102706,-0.001735,-0.002759,...,0.011052,-0.000168,-0.007371,0.0,0.005168,0.000008,-0.010703,0.0,0.007806,0.018102
9897,0.112756,0.223830,0.144039,0.051911,-0.013268,0.045539,0.0,0.036404,-0.003427,0.000996,...,-0.002160,0.000007,-0.004722,0.0,0.002365,0.000032,-0.007469,0.0,0.004606,0.007270
9898,0.058413,0.143470,0.139819,0.028420,-0.004275,0.015019,0.0,-0.001331,-0.003974,0.002768,...,0.003469,0.000148,-0.003495,0.0,0.001281,0.000019,-0.006464,0.0,0.003590,0.001747
